In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import *
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from IPython.display import display
from PIL import Image
import time
import tensorflow as tf


session_conf = tf.ConfigProto(
      intra_op_parallelism_threads=1,
      inter_op_parallelism_threads=1)
sess = tf.Session(config=session_conf)

K.set_session(sess)

t0 = time.time()

    # dimensions of our images.
img_width, img_height = 150, 150

train1_data_dir = 'data/train1'
train2_data_dir = 'data/train2'
train3_data_dir = 'data/train3'
val1_data_dir = 'data/val1'
val2_data_dir = 'data/val2'
val3_data_dir = 'data/val3'
nb_train_samples = 999
nb_validation_samples = 501
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model1 = Sequential()
model1.add(Conv2D(32, (3, 3), input_shape=input_shape))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model2 = Sequential()
model2.add(Conv2D(32, (3, 3), input_shape=input_shape))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model3 = Sequential()
model3.add(Conv2D(32, (3, 3), input_shape=input_shape))
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

mergedOut = Add()([model1.output,model2.output,model3.output])

mergedOut = Flatten()(mergedOut)    
mergedOut = Dense(64, activation='relu')(mergedOut)
mergedOut = Dropout(.5)(mergedOut)
mergedOut = Dense(2, activation='sigmoid')(mergedOut)

model = Model([model1.input,model2.input,model3.input], mergedOut)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

def generate_generator_multiple(generator,dir1, dir2, dir3, batch_size, img_height,img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    genX2 = generator.flow_from_directory(dir2,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    genX3 = generator.flow_from_directory(dir3,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            yield [X1i[0], X2i[0], X3i[0]], X3i[1]  #Yield both images and their mutual label
            
            
train_generator=generate_generator_multiple(generator=train_datagen,
                                           dir1=train1_data_dir,
                                           dir2=train2_data_dir,
                                           dir3=train3_data_dir,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_height)       


validation_generator=generate_generator_multiple(generator=test_datagen,
                                          dir1=val1_data_dir,
                                          dir2=val2_data_dir,
                                          dir3=val3_data_dir,
                                          batch_size=batch_size,
                                          img_height=img_height,
                                          img_width=img_height) 


model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('seq_try.h5')

t1 = time.time()

total = t1-t0
    
print(total)




Epoch 1/50
Found 666 images belonging to 2 classes.
Found 666 images belonging to 2 classes.
Found 666 images belonging to 2 classes.
61/62 [============================>.] - ETA: 1s - loss: 5.6637 - acc: 0.6311Found 332 images belonging to 2 classes.
Found 332 images belonging to 2 classes.
Found 332 images belonging to 2 classes.
62/62 [==============================] - 77s 1s/step - loss: 5.5721 - acc: 0.6371 - val_loss: 5.4086 - val_acc: 0.6626
Epoch 2/50
62/62 [==============================] - 78s 1s/step - loss: 4.4055 - acc: 0.5509 - val_loss: 0.7146 - val_acc: 0.4969
Epoch 3/50
62/62 [==============================] - 73s 1s/step - loss: 0.5888 - acc: 0.6351 - val_loss: 3.2984 - val_acc: 0.6025
Epoch 4/50
62/62 [==============================] - 70s 1s/step - loss: 0.8307 - acc: 0.7178 - val_loss: 1.7746 - val_acc: 0.4303
Epoch 5/50
62/62 [==============================] - 73s 1s/step - loss: 0.6788 - acc: 0.7703 - val_loss: 4.8046 - val_acc: 0.5369
Epoch 6/50
62/62 [=========